In [1]:
import os

from pyciemss.PetriNetODE.base import MiraPetriNetODESystem, ScaledBetaNoisePetriNetODESystem
from pyciemss.PetriNetODE.events import Event, StartEvent, LoggingEvent, ObservationEvent, StaticParameterInterventionEvent
import pyciemss

from pyciemss.PetriNetODE.interfaces import load_petri_model, setup_model, reset_model, intervene, sample, calibrate, optimize

In [2]:
MIRA_PATH = "test/models/evaluation_examples/scenario_1/"

filename = "scenario1_sir_mira.json"
filename = os.path.join(MIRA_PATH, filename)
model = ScaledBetaNoisePetriNetODESystem.from_mira(filename)
model


ScaledBetaNoisePetriNetODESystem(
	beta = Uniform(low: 0.09000000357627869, high: 0.10999999940395355),
	gamma = Uniform(low: 0.18000000715255737, high: 0.2199999988079071),
	pseudocount = 1
)

In [3]:


start_event = StartEvent(0.0, {"susceptible_population": 0.99, "infected_population": 0.01, "immune_population": 0.0})
model.load_event(start_event)

tspan = range(1, 10)
logging_events = [LoggingEvent(t) for t in tspan]
model.load_events(logging_events)

solution = model()

# See that the solution returns a dictionary where each value has length 10, one for each logging event.
assert len(solution["susceptible_population"]) == len(solution["infected_population"]) == len(solution["immune_population"]) == len(tspan)

solution

{'immune_population': tensor([0.0020, 0.0038, 0.0054, 0.0067, 0.0080, 0.0090, 0.0100, 0.0108, 0.0116]),
 'infected_population': tensor([0.0088, 0.0078, 0.0069, 0.0060, 0.0053, 0.0047, 0.0041, 0.0037, 0.0032]),
 'susceptible_population': tensor([0.9892, 0.9884, 0.9878, 0.9872, 0.9867, 0.9863, 0.9859, 0.9855, 0.9852])}

In [4]:
# Remove logging events
model.remove_logging_events()

# Add observations
observation_events = [ObservationEvent(1.1, {"susceptible_population": 0.9, "infected_population": 0.09, "immune_population": 0.01}), 
                      ObservationEvent(2.1, {"susceptible_population": 0.8, "infected_population": 0.18, "immune_population": 0.02}),
                      ObservationEvent(3.1, {"susceptible_population": 0.7, "infected_population": 0.27, "immune_population": 0.03}),
                      ObservationEvent(4.1, {"susceptible_population": 0.6, "infected_population": 0.36, "immune_population": 0.04})]

model.load_events(observation_events)
model

ScaledBetaNoisePetriNetODESystem(
	beta = Uniform(low: 0.09000000357627869, high: 0.10999999940395355),
	gamma = Uniform(low: 0.18000000715255737, high: 0.2199999988079071),
	pseudocount = 1
)

In [5]:
model._static_events

[StartEvent(time=0.0, initial_state={'susceptible_population': tensor(0.9900), 'infected_population': tensor(0.0100), 'immune_population': tensor(0.)}),
 ObservationEvent(time=1.100000023841858, observation={'susceptible_population': tensor(0.9000), 'infected_population': tensor(0.0900), 'immune_population': tensor(0.0100)}),
 ObservationEvent(time=2.0999999046325684, observation={'susceptible_population': tensor(0.8000), 'infected_population': tensor(0.1800), 'immune_population': tensor(0.0200)}),
 ObservationEvent(time=3.0999999046325684, observation={'susceptible_population': tensor(0.7000), 'infected_population': tensor(0.2700), 'immune_population': tensor(0.0300)}),
 ObservationEvent(time=4.099999904632568, observation={'susceptible_population': tensor(0.6000), 'infected_population': tensor(0.3600), 'immune_population': tensor(0.0400)})]

In [6]:
# Show that inference works.

from pyro.infer.autoguide import AutoNormal
from pyro.poutine import block
from pyro.infer import SVI, Trace_ELBO
from pyro.optim import Adam
import pyro

guide = AutoNormal(model)
optim = Adam({'lr': 0.03})
loss_f = Trace_ELBO(num_particles=1)
verbose = True

svi = SVI(model, guide, optim, loss=loss_f)

pyro.clear_param_store()

for j in range(100):
    # calculate the loss and take a gradient step
    # Passing a data argument to svi.step() will condition the model on the data.
    loss = svi.step()
    if verbose:
        if j % 25 == 0:
            print("[iteration %04d] loss: %.4f" % (j + 1, loss))

[iteration 0001] loss: 36.9257
[iteration 0026] loss: 35.5110
[iteration 0051] loss: 34.3351
[iteration 0076] loss: 30.4136


In [7]:
# Remove the observation events
model.remove_observation_events()

In [8]:
# Load some static parameter intervention events
model.load_event(StaticParameterInterventionEvent(2.99, "beta", 0.0))
model.load_event(StaticParameterInterventionEvent(4.11, "beta", 10.))

# Load the logging events again
model.load_events(logging_events)

In [9]:
model._static_events

[StartEvent(time=0.0, initial_state={'susceptible_population': tensor(0.9900), 'infected_population': tensor(0.0100), 'immune_population': tensor(0.)}),
 LoggingEvent(time=1),
 LoggingEvent(time=2),
 StaticParameterInterventionEvent(time=2.990000009536743, parameter=beta, value=0.0),
 LoggingEvent(time=3),
 LoggingEvent(time=4),
 StaticParameterInterventionEvent(time=4.110000133514404, parameter=beta, value=10.0),
 LoggingEvent(time=5),
 LoggingEvent(time=6),
 LoggingEvent(time=7),
 LoggingEvent(time=8),
 LoggingEvent(time=9)]

In [10]:
model.G.parameters

{'beta': <mira.modeling.ModelParameter at 0x7fd4d84fca00>,
 'gamma': <mira.modeling.ModelParameter at 0x7fd4d84fc970>}

In [11]:
from pyciemss.PetriNetODE.base import get_name
[get_name(p) for p in model.G.parameters.values()]
# model.G.parameters

['beta', 'gamma']

In [12]:
model._observation_var_names
model()

{'immune_population': tensor([0.0018, 0.0035, 0.0051, 0.0064, 0.0743, 0.2350, 0.3682, 0.4782, 0.5690]),
 'infected_population': tensor([0.0092, 0.0084, 0.0077, 0.0064, 0.8972, 0.7650, 0.6318, 0.5218, 0.4310]),
 'susceptible_population': tensor([ 9.8900e-01,  9.8808e-01,  9.8725e-01,  9.8725e-01,  2.8518e-02,
          6.4241e-06,  6.1728e-09, -6.7957e-11,  3.2436e-11])}

In [13]:
# use pyro predictive
from pyro.infer import Predictive

# Get the return value of model.
predictions = Predictive(model, guide=guide, num_samples=10)()

In [14]:
predictions

{'gamma': tensor([0.1935, 0.2000, 0.1992, 0.1947, 0.2112, 0.1946, 0.1876, 0.2133, 0.2017,
         0.1986]),
 'immune_population_sol': tensor([[0.0971, 0.2558, 0.3867, 0.4946, 0.5835, 0.6568, 0.7171, 0.7669, 0.8079],
         [0.1001, 0.2631, 0.3967, 0.5060, 0.5955, 0.6689, 0.7289, 0.7780, 0.8183],
         [0.0997, 0.2622, 0.3954, 0.5046, 0.5941, 0.6674, 0.7275, 0.7767, 0.8170],
         [0.0976, 0.2571, 0.3886, 0.4967, 0.5857, 0.6590, 0.7193, 0.7690, 0.8099],
         [0.1053, 0.2755, 0.4134, 0.5250, 0.6155, 0.6887, 0.7479, 0.7959, 0.8348],
         [0.0976, 0.2571, 0.3885, 0.4966, 0.5856, 0.6589, 0.7192, 0.7689, 0.8097],
         [0.0944, 0.2492, 0.3776, 0.4841, 0.5723, 0.6455, 0.7061, 0.7564, 0.7981],
         [0.1062, 0.2778, 0.4165, 0.5286, 0.6191, 0.6923, 0.7514, 0.7991, 0.8377],
         [0.1009, 0.2650, 0.3993, 0.5090, 0.5987, 0.6720, 0.7320, 0.7809, 0.8209],
         [0.0995, 0.2616, 0.3946, 0.5036, 0.5931, 0.6664, 0.7265, 0.7757, 0.8161]]),
 'infected_population_sol': tensor